<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Single-Record Ingestion for Ranking With Relevance Score

This example walks through the Arize single-record API for [ingesting ranking data with support for NDCG only](https://docs.arize.com/arize/model-types/ranking#case-1-ranking-with-relevance-score). Guides for other model types are available [here](https://docs.arize.com/arize/sending-data-to-arize/model-types).

## Install and Import Dependencies

In [ ]:
!pip install -q arize==6.1.0

import uuid
from datetime import datetime


import pandas as pd
from arize.api import Client
from arize.utils.types import Environments, ModelTypes, Schema, RankingPredictionLabel, RankingActualLabel

## Download and Display Data
For this tutorial, we'll use a sample Parquet file that represents a model that predicts the likelihood that a user clicks on a recommended hotel in an ordered list.

**Note**: This tutorial is for record-by-record ingestion, use the sample table as a visual indicator of how each column logs into Arize. 

In [ ]:
file_url = "https://storage.googleapis.com/arize-assets/documentation-sample-data/data-ingestion/ranking-assets/ranking-relevance-score-sample-data.parquet"
df = pd.read_parquet(file_url)
df.head()

## Create Arize Client

Sign up/ log in to your Arize account [here](https://app.arize.com/auth/login). Find your [space and API keys](https://docs.arize.com/arize/sending-data/sdk-reference/python-sdk/arize.init#retrieving-space-and-api-keys). Copy/paste into the cell below.

In [ ]:
SPACE_KEY = "YOUR SPACE KEY"  # Change this line.
API_KEY = "YOUR API KEY"  # Change this line.

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ CHANGE SPACE AND API KEYS")
else:
    print("✅ Arize client setup done! Now you can start using Arize!")

## Define Features



In [ ]:
features = {
    "prop_log_historical_price" : 5.22 ,
    "price_usd" : 98.04,
    "promotion_flag" : 1.0,
    "search_destination_id" : 8192.0,
    "search_length_of_stay" : 4.0,
    "search_booking_window" : 227.0,
    "search_adults_count" : 1.0,
    "search_children_count" : 0.0,
    "search_room_count" : 1.0,
    "search_saturday_night_bool" : 0.0,
    "destination" : "Yellowstone",
    }

##Define Prediction & Actual Labels

Create ranking labels using the arguments: `group_id`, `rank`, `score`, and `relevance_scores` before logging the record. 

In [ ]:
pred_label = RankingPredictionLabel(
      group_id="148", # search_id column
      rank=1, # rank column
      score=0.155441 # predicted_relevancy_score coulmn
  )
act_label = RankingActualLabel(
    relevance_score=0 # actual_relevancy_score column
)

## Log Data To Arize

Log the record using the [single-record API](https://docs.arize.com/arize/sending-data-to-arize/data-ingestion-methods/sdk-reference/python-sdk/arize.log)

In [ ]:
response = arize_client.log(
    model_id="demo-ranking-with_relevance-score",
    model_version="v1",
    environment=Environments.PRODUCTION,
    model_type=ModelTypes.RANKING,
    prediction_id="311103e3-a493-40ea-a21a-e457d617c956",
    prediction_label=pred_label,
    actual_label=act_label,
    features=features,
)
## Listen to response code to ensure successful delivery
result = response.result()

if result.status_code == 200:
    print(f"✅ Successfully logged data to Arize!")
else:
    print(f'Log failed with response code {res.status_code}, {res.text}')